# NLP

# 3. Transformers: hype

<img src="attachements/third/nlp_before.png" width="1000"/>

## План занятий

### 1. ЧБДТ: Что Было До Трансформеров
   1. При чём тут нейро-лингвистическое программирование?
   2. Почему вектора?
   3. Счётные методы: CountVectorizer, TF-IDF, добавляем контекст 
   4. Дистрибутивная семантика: Word2vec, FastText
   5. Рекуррентные нейронные сети: LSTM, GRU
### 2. Трансформеры: база
   1. Мотивация
   2. Attention из all you need?: виды attention, интуиция, реализация
   3. Архитектура Transformer: эмбеддинги, энкодер, декодер
### 3. Трансформеры: на волне хайпа
   1. BERT: pre-training, fine-tuning, семейка
   2. GPT: language modelling, n-shot, prompt-engineering, decoding stratetiges
   3. T5: multitask models, seq2seq

## Transformer: архитектура

Исходно Transformer - Encoder-Decoder архитектура для seq2seq задач (машинного перевода).

Задачи в NLP бывают разные, для многих задач архитектура Encoder-Decoder избыточна.

Рассмотрим три основных семейства моделей:
- Encoder (BERT-family) - NLU (natural language understanding) задачи
- Decoder (GPT-family) - NLG (natural language generation) задачи 
- Encoder-Decoder (T5 & co.) - Text2Text задачи

<img src="attachements/third/transformer-arch.png" width="1000"/>

Две стадии: pretraining + finetuning

## BERT: Bidirectional Encoder Representations from Transformers

BERT - encoder-only transformer. Основная задача: генерировать контекстуализированные эмбеддинги для слов и предложения.

Проблемы прошлых моделей:
- Омонимия - значения слов зависят от контекста
- Out-of-vocabulary слова - новые/редкие слова, опечатки
- "Статичность" эмбеддингов

<img src="attachements/third/bert_embeddings.png" width="700"/>

## Pre-training

Идея из word2vec: получить знания о структуре и смысле языка, используя огромные массивы неразмеченных данных.

### Masked language modeling

Похоже на CBOW подход из word2vec.

<img src="attachements/third/cbow.png" width="800"/>

Основные отличия - учитываем связь между словами и весь контекст предложения.

Задача: предсказать слово, которое было скрыто маской на основе всего контекста предложения.

Пайплайн:
- Выбираем 15% токенов
  - в 80% случаев заменяем на токен [MASK]
  - в 10% случаев заменяем на рандомный
  - в 10% случаев оставляем без изменений
- Предсказываем, какой токен был заменен
  - Классификация по всему словарю

<img src="attachements/third/mlm.png" width="1000"/>

### Next sentence prediction

Кроме эмбеддингов слов, мы также получить хороший эмбеддинг предложения. Можно просто усреднить (как мы делали для word2vec), но всегда ли усреднять хорошо?

Задача - предсказать, идут ли предложения друг за другом в тексте?
- Предсказание делается от специального токена [CLS]
- Предложения разделены специальным токеном [SEP]
- 50% предложения следуют друг за другом
- 50% предложения независимы (выбираем случайное из выборки)

<img src="attachements/third/nsp.png" width="1000"/>

## Fine-Tuning

Доучиваем модельку под различные задачи на небольших данных.

### Задачи

#### Classification

Одно предложение - определяем тональность, токсичность или тематику текста.

Пара предложений - определяем связь между предложениями. Например, подходит ли кандидат на вакансию.

##### NLI - Natural Language Inference

Понимаем логическую связь предложений: продолжает, отрицает или не связано.

<img src="attachements/third/nli_task.png" width="800"/>

#### Sequence labelling

Необходимо классифицировать каждый токен в предложении.

Самая частая задача - Named Entity Recognition. Иногда использую как нарицательное.

<img src="attachements/third/ner_task.png" width="800"/>

#### Text segmentation

Задача - выделить границы интересующего нас сегмента текста.

<img src="attachements/third/text_segmentation.png" width="700"/>

#### Как учим BERT на эти задачи?

<img src="attachements/third/bert-tasks.png" width="900"/>

### Токенизация

Три типа токенов:
- Позиционные эмбеддинги - для учета порядка слов
- Эмбеддинги сегментов - для разделения предложений
- Эмбеддинги токенов - привычный слой эмбеддингов

<img src="attachements/third/bert_tokenization.png" width="1000"/>

А в остальном - буквально тот же самый энкодер из Transformer'a.

### Семейка BERT

<table>
<thead>
  <tr>
    <th>Модель</th>
    <th>Саммари</th>
    <th>Описание</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>BERT</td>
    <td>База</td>
    <td>Задачи:<br>- MLM (static masking)<br>- NSP</td>
  </tr>
  <tr>
    <td>RoBERTa</td>
    <td>Улучшенный процесс обучения</td>
    <td>Особенности:<br>- Ооочень много данных (160Gb vs 16Gb)<br>- Большой размер батча (8000 vs 256)<br><br>Задачи:<br>- MLM (dynamic masking)<br>- No NSP</td>
  </tr>
  <tr>
    <td>Electra</td>
    <td>Улучшенный процесс обучения</td>
    <td>Особенности:<br>- Тренинг на всех данных, а не только маскированных<br><br>Задачи:<br>- Replaced token detection (instead of MLM)<br>- No NSP</td>
  </tr>
  <tr>
    <td>Albert</td>
    <td>Облегченный BERT</td>
    <td>Особенности:<br>- Шэринг параметров между слоями (FF / FF+MHA / MHA)<br>- Факторизация эмбеддингов<br><br>Задачи:<br>- MLM<br>- SOP (sentence order prediction)</td>
  </tr>
  <tr>
    <td>DistillBERT &amp; TinyBERT</td>
    <td>Маленькие BERT'ы, полученные за счет дистилляции</td>
    <td>Особенности:<br>- DistillBERT учится предсказывать предсказания большого BERT<br>- TinyBERT учится предсказывать в т.ч. промежуточные слои <img src="attachements/third/distill_vs_tiny.png" width="500"/> </td>
  </tr>
</tbody>
</table>

### Кодим!

#### База

In [1]:
import random
import numpy as np
import torch

def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
seed_everything(0)

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

#### Читаем и готовим датасет - это мы уже видели

In [4]:
with open("data/dataset.txt")as file:
    data = file.read().split("\n")[:-1]
    data = list(map(lambda x: x.split(" ", maxsplit=1), data))
    data = pd.DataFrame(data, columns=["target", "text"])

In [5]:
data.head()

,target,text
0,__label__INSULT,скотина! что сказать
1,__label__NORMAL,я сегодня проезжала по рабочей и между домами ...
2,__label__NORMAL,очередной лохотрон. зачем придумывать очередно...
3,__label__NORMAL,"ретро дежавю ... сложно понять чужое сердце , ..."
4,__label__NORMAL,а когда мы статус агрогородка получили?


In [6]:
data["target"].value_counts()

target
__label__NORMAL                                       203685
__label__INSULT                                        28567
__label__INSULT,__label__THREAT                         6317
__label__THREAT                                         5460
__label__OBSCENITY                                      2245
__label__INSULT,__label__OBSCENITY                      1766
__label__INSULT,__label__OBSCENITY,__label__THREAT       176
__label__OBSCENITY,__label__THREAT                        74
Name: count, dtype: int64

In [7]:
data["target"] = data["target"] \
    .apply(lambda x: "__label__OBSCENITY" if "OBSCENITY" in x else x) \
    .apply(lambda x: "__label__THREAT" if "THREAT" in x else x)

In [8]:
data["target"].value_counts()

target
__label__NORMAL       203685
__label__INSULT        28567
__label__THREAT        11777
__label__OBSCENITY      4261
Name: count, dtype: int64

In [9]:
data = data.sample(frac=1).groupby("target").head(6666).reset_index(drop=True)

In [10]:
data["target"].value_counts()

target
__label__INSULT       6666
__label__NORMAL       6666
__label__THREAT       6666
__label__OBSCENITY    4261
Name: count, dtype: int64

In [11]:
data["target"] = data["target"].factorize()[0]

In [12]:
data.head()

,target,text
0,0,"это твоя мама толстая все сжирает,а у путьки н..."
1,1,правда на 100% . прежде чем что то требовать н...
2,0,"все правильно не уступай трамваю, и поездам то..."
3,2,"таких людей убивать нужно, бедный ребёнок, сер..."
4,1,"опять театр опять развлекуха!! опять клоуны, а..."


In [13]:
data["target"].value_counts()

target
0    6666
1    6666
2    6666
3    4261
Name: count, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_data, test_data = train_test_split(data, test_size=0.15, stratify=data["target"])

In [16]:
X_train, X_test = train_data["text"].values, test_data["text"].values 
y_train, y_test = train_data["target"].values, test_data["target"].values

#### Строим модель!

Huggingface - главная библиотечка для взаимодействия с трансформерами.

Тут есть:
- Хаб предобученных моделей
- Вспомогательные утилиты для обучения и оценки моделей
- И чего только нет...

<img src="attachements/third/huggingface_models.png" width="1000"/>

In [17]:
from transformers import BertTokenizerFast, BertForSequenceClassification

In [18]:
BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LR = 1e-4
NUM_EPOCHS = 3

Берём BERT для классификации последовательностей.

Всё, нужное нам, уже инициализированно. Не забываем файн-тюне.

In [19]:
model = BertForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=data["target"].nunique())
tokenizer = BertTokenizerFast.from_pretrained("cointegrated/rubert-tiny2")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model.to(DEVICE)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, 

#### Токенизатор и вызов модели

##### Токенизатор

In [21]:
tokenizer("я просто в тебя втюрилась")

{'input_ids': [2, 343, 9922, 314, 29852, 314, 24174, 52655, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
tokenizer.decode([2, 343, 9922, 314, 29852, 314, 24174, 52655, 3])

'[CLS] я просто в тебя втюрилась [SEP]'

In [23]:
tokenizer("я просто в тебя втюрилась", "потому что дора дура")

{'input_ids': [2, 343, 9922, 314, 29852, 314, 24174, 52655, 3, 14665, 1046, 745, 1232, 47802, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
tokenizer.decode([2, 343, 9922, 314, 29852, 314, 24174, 52655, 3, 14665, 1046, 745, 1232, 47802, 3],)

'[CLS] я просто в тебя втюрилась [SEP] потому что дора дура [SEP]'

In [25]:
tokenizer(["я просто в тебя втюрилась", "потому что дора дура"], padding=True, return_tensors="pt")

{'input_ids': tensor([[    2,   343,  9922,   314, 29852,   314, 24174, 52655,     3],
        [    2, 14665,  1046,   745,  1232, 47802,     3,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]])}

#### Вызов модели

In [26]:
model(**tokenizer("я просто в тебя втюрилась", return_tensors="pt"))

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0040, -0.0287, -0.0198,  0.0193]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [27]:
model(**tokenizer("я просто в тебя втюрилась", return_tensors="pt"), output_hidden_states=True)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0040, -0.0287, -0.0198,  0.0193]], grad_fn=<AddmmBackward0>), hidden_states=(tensor([[[-0.0142,  0.0298, -0.0084,  ...,  0.0131,  0.0028,  0.0183],
         [-0.9121,  0.3245, -0.3886,  ...,  0.6066,  0.6993, -0.5504],
         [ 1.3372,  0.3197, -0.4329,  ..., -0.1644, -0.7971,  0.4067],
         ...,
         [ 0.9813, -1.7365, -0.0614,  ...,  1.5010, -0.3529,  0.9264],
         [-1.0524, -0.9197,  2.0454,  ..., -0.1242, -0.9972,  1.4023],
         [ 0.1239, -0.2321, -0.0834,  ..., -0.5896, -1.6422, -0.1138]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[-0.0365,  0.1742,  0.2868,  ..., -0.0316,  0.0761, -0.5213],
         [-0.5092, -0.2252,  0.0681,  ..., -0.0459,  0.5087, -0.8478],
         [ 1.8197, -0.0455, -0.6360,  ...,  0.1347, -0.4981, -0.3415],
         ...,
         [ 1.4533, -1.4143, -0.2475,  ...,  1.0662,  0.3260,  0.2414],
         [-0.8736, -1.2774,  1.3637,  ..., -0.3602, -0.5690,  0.6849],
         [-0.

In [28]:
model(**tokenizer("я просто в тебя втюрилась", return_tensors="pt"), labels=torch.tensor(0))

SequenceClassifierOutput(loss=tensor(1.3762, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0040, -0.0287, -0.0198,  0.0193]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

#### Датасет и даталоадер

In [29]:
from torch.utils.data import Dataset, DataLoader
from operator import itemgetter

In [30]:
class BertDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [31]:
def collate(batch):
    texts = list(map(itemgetter(0), batch))
    labels = list(map(itemgetter(1), batch))

    tokens = tokenizer(texts, return_tensors="pt", max_length=64, padding="max_length", truncation=True)
    labels = torch.tensor(labels)

    return tokens, labels

In [32]:
train_dataset = BertDataset(X_train, y_train)
test_dataset = BertDataset(X_test, y_test)

In [33]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate)

#### Тренинг

In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score

def compute_metrics(y_true, y_pred, average='macro'):
    precision = precision_score(y_true, y_pred, average=average)
    recall = recall_score(y_true, y_pred, average=average)
    f1 = f1_score(y_true, y_pred, average=average)

    print(f'{average.capitalize()} Precision = {precision:.4f}, Recall = {recall:.4f}, F1 = {f1:.4f}')

    return precision, recall, f1

In [36]:
def train_epoch():
    model.train()

    losses = []
    for tokens, labels in tqdm(train_dataloader, total=len(train_dataloader)):
        tokens = tokens.to(DEVICE)
        labels = labels.to(DEVICE)

        loss = model(**tokens, labels=labels).loss

        loss.backward()
        losses.append(loss.item())
        optimizer.step()

        optimizer.zero_grad()

    epoch_loss = sum(losses) / len(losses)

    return epoch_loss

In [37]:
@torch.no_grad()
def test_epoch():
    model.eval()

    losses = []
    predictions = []
    for tokens, labels in tqdm(test_dataloader, total=len(test_dataloader)):
        tokens = tokens.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(**tokens, labels=labels)

        pred_labels = outputs.logits.argmax(dim=1).tolist()

        predictions.extend(pred_labels)
        losses.append(outputs.loss.item())

    epoch_loss = sum(losses) / len(losses)
    
    return epoch_loss, predictions

In [38]:
def train_model():
    for epoch in range(NUM_EPOCHS):
        train_loss = train_epoch()
        test_loss, pred_test = test_epoch()

        print(f"TRAIN Epoch {epoch+1}, Loss: {train_loss:.4f}")
        print(f"TEST Epoch {epoch+1}, Loss: {test_loss:.4f}")

        compute_metrics(y_test, pred_test)
        print()
    
    return pred_test 

In [39]:
pred_test = train_model()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 0/162 [00:00<?, ?it/s]

100%|██████████| 29/29 [00:04<00:00,  5.91it/s]


TRAIN Epoch 1, Loss: 0.6297
TEST Epoch 1, Loss: 0.3823
Macro Precision = 0.8616, Recall = 0.8667, F1 = 0.8632



100%|██████████| 29/29 [00:06<00:00,  4.49it/s]


TRAIN Epoch 2, Loss: 0.2938
TEST Epoch 2, Loss: 0.3350
Macro Precision = 0.8824, Recall = 0.8756, F1 = 0.8784



100%|██████████| 29/29 [00:05<00:00,  5.65it/s]

TRAIN Epoch 3, Loss: 0.1888
TEST Epoch 3, Loss: 0.3361
Macro Precision = 0.8814, Recall = 0.8812, F1 = 0.8809



## GPT: Generative pre-trained transformer

### Language modeling

Language modeling - задача предсказания следующего токена (слова / подслова / буквы) в предложении.

По сути - генерация текста.

Примеры:
- Написание контента - сказок, рассказов, рекламных текстов
- Автодополнение (кода, поискового запроса, текста)

<img src="attachements/third/gpt-2-autoregression.gif" width="1000"/>

GPT - decoder-only transformer, задача которого - авторегрессионно генерировать следущее слово.

### Хитрости и особенности

- Очень много текстов
- Очень (ОЧЕНЬ!) большие модели

<img src="attachements/third/gpt_size.png" width="800"/>

#### Эмерджентность

Большие модели с большими данными обретают новые свойства: учатся решать задачи, которым специально не учились: перевод, ответы на вопросы, NER, сегментация текста, ...

И их больше не нужно файнтюнить!

#### Как заставить модель понять, что от неё нужно?

##### Zero-shot - просто попроси

<img src="attachements/third/zero_shot.png" width="700"/>

##### One-shot - покажи пример

<img src="attachements/third/one_shot.png" width="700"/>

##### Few-shot - покажи несколько примеров

<img src="attachements/third/few_shot.png" width="700"/>

#### Качество возрастает при увеличении модели и количества примеров

<img src="attachements/third/gpt_few_shot.png" width="1000"/>

#### Качество на отдельных задачах

Модель показывает себя очень хорошо либо даже опережает специализированные SOTA решения:

- LAMBADA - нужно предсказать последнее слово в предложении. (нужно улавливать глобальный контекст)
- StoryCloze - нужно выбрать верное окончание истории из четырех вариантов.
- HellaSwag - NLI датасет; нужно выбрать логически подходящее завершение текста.

<img src="attachements/third/gpt_pqrformance.png" width="700"/>

### Prompt-Engineering

Как ещё можно добиться, чтобы модель делала то, что вы от неё хотите?

<img src="attachements/third/prompt_engineering.png" width="600"/>

Подбор "затравки" - очень мощный инструмент. Моделями можно манипулировать!
- Я дам тебе 200 долларов чаевыми, если ты мне поможешь
- Let's think step-by-step
- ...

<img src="attachements/third/prompt_tuning.jpg" width="800"/>

Сейчас - это уже целый мир, всё ограничивается только фантазией.

### Кодим!

#### Читаем модель

In [40]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import json

In [41]:
model = GPT2LMHeadModel.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

In [42]:
VERBOSE = 5
BATCH_SIZE = 5
LR = 5e-5
NUM_EPOCHS = 2

In [43]:
model.to(DEVICE)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [44]:
def tokenize(texts):
    return tokenizer(texts, return_tensors="pt", padding=True)

In [45]:
def generate(prompt, do_sample=False, **kwargs):
    tokens = tokenize(prompt).to(DEVICE)
    out_tokens = model.generate(**tokens, do_sample=do_sample, **kwargs)

    return tokenizer.batch_decode(out_tokens, ignore_special_tokens=True)

In [46]:
generate("Шел медведь по лесу")

['Шел медведь по лесу,\n\t\tИ вдруг увидел он, что в лесу\n\t\t']

#### Читаем данные

In [47]:
with open("data/anecdotes.json") as file:
    anecdotes = json.load(file)

In [48]:
len(anecdotes)

39636

In [49]:
anecdotes[456]

'Включил я как-то Вечерний ургант выпуск 1290 от 17.04.2020 (на 15:00), а там армяне в нарды играют'

#### Чистим немножечко

In [50]:
import re

In [51]:
anecdotes = list(map(lambda x: re.sub(r"[\n\-\_\.]+\n", "\n", x), anecdotes))

#### Токенизируем немножечко

In [52]:
anecdote_tokens = tokenizer(anecdotes, max_length=2048)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [53]:
lengths = pd.Series(list(map(len, anecdote_tokens["input_ids"])))

In [54]:
lengths.describe(percentiles=[0.5, 0.95, 0.99])

count    39636.000000
mean       116.122944
std        176.377865
min          2.000000
50%         61.000000
95%        365.000000
99%        866.000000
max       2048.000000
dtype: float64

In [55]:
len(anecdotes)

39636

In [56]:
anecdotes = [anecdotes[i] for i in lengths[(lengths < 350) & (lengths > 4)].index]

Отсортируем, чтобы делать меньше паддингов и лишних вычислений.

Мы так уже делали, когда учили LSTM.

In [57]:
anecdotes = sorted(anecdotes, key=lambda x: -len(x))

In [58]:
anecdotes[-30:-28]

['Узбек-людоед съел пловца', '-У вас хлеб свежий?\n-Да.']

Нет ни GPUшки, ни бесконечности времени...

In [59]:
anecdotes = anecdotes[20000:20100]

#### Датасет и даталоадер

In [60]:
class AnecdoteDataset(Dataset):
    def __init__(self, anecdotes) -> None:
        self.anecdotes = anecdotes
    
    def __len__(self):
        return len(self.anecdotes)
    
    def __getitem__(self, index):
        return self.anecdotes[index]

In [61]:
def anecdote_collate(batch):
    return tokenize(batch)

In [62]:
anecdote_dataset = AnecdoteDataset(anecdotes)
anecdote_dataloader = DataLoader(anecdote_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=anecdote_collate)

#### А что там в этом батче?

In [63]:
next(iter(anecdote_dataloader))

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,    17,   563, 28506,  4351,   481,  2309, 32397,   432, 11548,
           541,  5981,    16,  7970, 20063, 43756,   291,   289, 21208,   225,
           203,    17,  9230,  2309,    35,  9398, 34210,    35, 47752,   280,
            16,   428,   694,  7757,   282,  2218,   914, 32225],
        [    0,     0,     0,     0,   983,  4194,  8327, 17398, 17889,  3417,
         12380,    30,   203, 45652,  2796,   289,   727,  2679,   297,  4351,
          6858,   309,  7812,    16,   515,   457,  1849,   448, 16823,   475,
         44927,   414,    16,   289, 11900,  2796,  3619, 19926,   334, 13503,
            16,   367,   727,  2679,   297,  4351,   384, 18758,   673,   272,
           203,  1009,   262,  3484,  1514,  1483,   436,  1960],
        [   17,   629, 44098, 10989,   299, 18025,  2688,  9880,

Заметим:
1. Паддинг слева
2. Паддится не так много (но можно и меньше: как?)

#### Обучение

In [64]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [65]:
def train_epoch(epoch):
    model.train()
    losses = []

    for i, input_tokens in enumerate(tqdm(anecdote_dataloader, total=len(anecdote_dataloader))):
        input_tokens = input_tokens.to(DEVICE)

        labels = input_tokens["input_ids"].clone()
        labels[labels == 0] = -100  # to ignore them when computing loss
        
        loss = model(
            **input_tokens,
            labels=labels,
            return_dict=True
        ).loss

        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        if (i + 1) % VERBOSE == 0:
            print(f"Step {i + 1}: loss - {sum(losses[-VERBOSE:-1]) / VERBOSE}")
            model.eval()
            print(generate("Шел медведь по лесу"))
            model.train()

    epoch_loss = sum(losses) / len(losses)

    print(f"EPOCH {epoch} - epoch loss: {(epoch_loss):3f}")
    torch.cuda.empty_cache()

    return epoch_loss

In [66]:
for i in range(NUM_EPOCHS):
    train_epoch(i)

 20%|██        | 4/20 [00:03<00:14,  1.12it/s]

Step 5: loss - 3.9486947059631348


 25%|██▌       | 5/20 [00:04<00:14,  1.02it/s]

['Шел медведь по лесу.  Он увидел, что на дереве сидит медведь.  Он схватил его']


 45%|████▌     | 9/20 [00:08<00:09,  1.20it/s]

...

['Шел медведь по лесу. Вдруг видит — на сосне сидит мужик.\n— Что это ты']


 95%|█████████▌| 19/20 [00:15<00:00,  1.32it/s]

Step 20: loss - 2.601138877868652


100%|██████████| 20/20 [00:16<00:00,  1.21it/s]

['Шел медведь по лесу. Вдруг видит — на сосне сидит мужик и спрашивает:\n— ']
EPOCH 1 - epoch loss: 3.199173


In [67]:
model.eval();

##### Decoding strategies

Трейдофф между связностью и разнообразием.

##### Greedy decoding

На каждом шаге генерации выбираем самый вероятный токен.

Получаем:
- очень связный текст
- невозможность получения разнообразных последовательностей

<img src="attachements/third/greedy_decoding.webp" width="600"/>

In [68]:
generate("Шел медведь по лесу")

['Шел медведь по лесу. Вдруг видит — на сосне сидит мужик и спрашивает:\n— ']

##### Sampling

##### Top-k sampling

Случайно выбираем из первых К предложенных вариантов согласно предсказанным вероятностям токенов.

<img src="attachements/third/topk_sampling.webp" width="600"/>

In [69]:
generate("Шел медведь по лесу", do_sample=True, top_k=3)

['Шел медведь по лесу и вдруг видит:\n- ААААААААААА!\n-']

##### Softmax temperature

Способ сделать распределение более равномерным.

Таким образом, выбор в Top-k семплинге будет более разнообразный.

<img src="attachements/third/softmax_temperature.jpg" width="800"/>

Чам больше температура, тем более равномерное распределение получается.

In [70]:
generate("Шел медведь по лесу", do_sample=True, top_k=100, temperature=2.)

['Шел медведь по лесу&hellip; И вдруг к одному мальчику. Ну все равно,']

##### Проблемка

Хочется сделать К динамическим: иногда вариантов очень много, а иногда - мало.

<img src="attachements/third/topk_problem.png" width="600"/>

##### Top-p sampling

К зависит от суммы вероятностей К предсказанных токенов.

<img src="attachements/third/top_p_sampling.webp" width="600"/>

In [71]:
generate("Шел медведь по лесу", do_sample=True, top_p=0.7, temperature=2.)

['Шел медведь по лесу, \nИ вдруг из леса навстречу ему\nЛеопард выходит\n']

##### Beam search

Прошлые методы выбирают следующий токен без учета контекста.

Beam search - строит несколько вариантов генераций и выбирает самую вероятную *последовательность* токенов.

<img src="attachements/third/beam_search.webp" width="800"/>

Это требует больше вычислений, но результаты получаются намного лучше.

In [72]:
generate(
    "Шел медведь по лесу", 
    num_beams=4,
    no_repeat_ngram_size=2,
    num_return_sequences=3,
)

['Шел медведь по лесу, а навстречу ему заяц. \n- Здравствуй, медведь! ',
 'Шел медведь по лесу, а навстречу ему заяц. \n- Ты кто такой? ',
 'Шел медведь по лесу, а навстречу ему заяц. \n- Ты кто такой?\n']

In [73]:
generate(
    "Шел медведь по лесу", 
    max_length=64, 
    num_beams=4,
    no_repeat_ngram_size=2,
    num_return_sequences=3,
    do_sample=True,
    top_p=0.73,
    temperature=2.
)

['Шел медведь по лесу. \nМедведь и говорит:\n- У меня есть ружье. Я хочу, чтобы ты меня пострелял.\nБандит обрадовался. \n\nИ говорит ему: "Блядь, а я тебя поубивал, блядь"\nМохнатый отвечает:',
 'Шел медведь по лесу. \nМедведь и говорит:\n- У меня есть ружье. Я хочу, чтобы ты меня пострелял.\nБандит обрадовался. \n\nИ говорит ему: "Блядь, а я тебя поубивал, блядь"\nМохнатый: ',
 'Шел медведь по лесу. \nМедведь и говорит:\n- У меня есть ружье. Я хочу, чтобы ты меня пострелял.\nБандит обрадовался. \n\nИ говорит ему: "Блядь, а я тебя поубивал, блядь"\nМохнатый думает:']

## T5: Transfer learning in all its glory

А что если объединить BERT и GPT и решать оооочень много различных задач?

Получится T5 - Encoder-Decoder transformer, предназначенный для решения text2text задач.

Можно представлять различные задачи в виде Text2text:

<img src="attachements/third/t5_multitask.png" width="1000"/>

### Предобучение

Почти как BERT, только сложнее: маскируем последовательности слов.

<img src="attachements/third/t5_pretraining.png" width="1000"/>

### Обучение

Задачи:
- Перевод - переводим с одного языка на другой.
- Суммаризация - делаем краткий пересказ текста.
- Question answering - отвечаем на вопрос по тексту.
- Классификация текстов - относим текст к какой-то группе.
- Генерация текстов - сочиняем (продолжаем) текст.
- Регрессия по текстам - предсказываем число по тексту (анализ тональности, возраст, ...). 
- NLI (Natural language inference) - понимаем логическую связь предложений: продолжает, отрицает или не связано.
- Сегментация текстов - находим отрезки текста, где что-то важное нам происходит (аспектный анализ тональности, ответ на вопрос, ...)

*(Т5 училась не на все эти задачи, некоторые просто как пример.)*

<img src="attachements/third/t5_tasks.png" width="1000"/>

### Encoder-Decoder vs Decoder-only

- Но ведь GPT умеет делать то же самое!
- Да, что-то из этого может и BERT делать...

<img src="attachements/third/transformer-arch.png" width="1000"/>

#### Encoder-only

Всякие BERT, RoBERTa and co.

Любые NLU-only задачи:
- Text Classification
- Sequence labelling
- Sentiment analysis
- NLI
- QA
- Search & Recommender systems (спойлер!)

#### Decoder-only

Всякие GPT, XLNet and co.

Любые чисто генеративные задачи:
- Text completion
- Text generation
- Creative writing

И задачи, где не требуется глубокое понимание:
- диалоговые системы

#### Encoder-Decoder

Всякие Transformer, BART, T5 and co.

Задачи на понимание + генерацию:
- Machine translation
- Summarization
- QA

#### Реальность

Берем ChatGPT и погнали...

### Кодим!

In [1]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from datasets import load_dataset

/Users/aleksiej/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = T5TokenizerFast.from_pretrained("cointegrated/rut5-small")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/aleksiej/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
You are using a

У huggingface, кроме моделей, есть и датасеты.

<img src="attachements/third/huggingface_datasets.png" width="1000"/>

In [3]:
datasets = load_dataset('IlyaGusev/gazeta', revision="v2.0", split=['train[0:100]', 'validation[0:100]'])
datasets

[Dataset({
     features: ['text', 'summary', 'title', 'date', 'url'],
     num_rows: 100
 }),
 Dataset({
     features: ['text', 'summary', 'title', 'date', 'url'],
     num_rows: 100
 })]

In [4]:
datasets[0][0]

{'text': 'Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: согласно Налоговому кодексу, базовый тариф, установленный правительством, может быть уменьшен в пять раз или увеличен до 10 раз. Сборы идут в региональные бюджеты, откуда растекаются на общие нужды. Транспортный налог — один из основных источников бюджетных доходов — предлагается направить исключительно на дорожные фонды. Так, автомобилисты будут понимать, за что они платят, а дорожники будут иметь гарантированный доход. Кроме налога дорожные фонды будут пополняться за счет бюджетных средств и проезда по платным дорогам. Более того, транспортный налог предлагается завуалировать в акцизы на бензин. Привычную и раздражающую систему ежегодной оплаты квитанции предлагается изменить, включив налог в стоимость топлива. Минэкономразвития говорит об удвоении акцизы, которая сегодня состав

In [5]:
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

In [6]:
def preprocess_data(examples):
    model_inputs = tokenizer(examples["text"], max_length=MAX_INPUT_LENGTH, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["title"], 
            max_length=MAX_TARGET_LENGTH, 
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [7]:
datasets[0] = datasets[0].map(preprocess_data, batched=True)
datasets[1] = datasets[1].map(preprocess_data, batched=True)

In [8]:
datasets

[Dataset({
     features: ['text', 'summary', 'title', 'date', 'url', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 100
 }),
 Dataset({
     features: ['text', 'summary', 'title', 'date', 'url', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 100
 })]

In [9]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

<img src="attachements/third/seq2seq_arguments.png" width="800"/>

In [10]:
args = Seq2SeqTrainingArguments(
    "t5_small_absum",
    evaluation_strategy="steps",
    eval_steps=20,
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="steps",
    save_steps=20,
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    report_to=None
)

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [12]:
trainer = Seq2SeqTrainer(
    model_init=lambda: T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small"),
    args=args,
    train_dataset=datasets[0],
    eval_dataset=datasets[1],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [13]:
trainer.train()

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
 27%|██▋       | 20/75 [00:29<01:18,  1.43s/it]

{'loss': 3.8417, 'learning_rate': 7.333333333333333e-05, 'epoch': 0.8}


                                               
 27%|██▋       | 20/75 [00:36<01:18,  1.43s/it]

{'eval_loss': 3.26416015625, 'eval_runtime': 7.2146, 'eval_samples_per_second': 13.861, 'eval_steps_per_second': 3.465, 'epoch': 0.8}


 53%|█████▎    | 40/75 [01:06<00:46,  1.34s/it]

...

                                               
 80%|████████  | 60/75 [01:46<00:19,  1.33s/it]

{'eval_loss': 3.118772268295288, 'eval_runtime': 5.5099, 'eval_samples_per_second': 18.149, 'eval_steps_per_second': 4.537, 'epoch': 2.4}


100%|██████████| 75/75 [02:08<00:00,  1.71s/it]

{'train_runtime': 128.5476, 'train_samples_per_second': 2.334, 'train_steps_per_second': 0.583, 'train_loss': 3.3105093892415365, 'epoch': 3.0}


TrainOutput(global_step=75, training_loss=3.3105093892415365, metrics={'train_runtime': 128.5476, 'train_samples_per_second': 2.334, 'train_steps_per_second': 0.583, 'train_loss': 3.3105093892415365, 'epoch': 3.0})

Считаем последний чекпоинт

In [14]:
model_dir = "t5_small_absum/checkpoint-60"

tokenizer = T5TokenizerFast.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)

Попробуем суммаризовать какой-нибудь текст

In [15]:
text_to_summarize = datasets[1][0]["text"]
text_to_summarize

'В 2020 году инфляция в России составит 3,5-4%, прогнозирует Центробанк. В первом квартале она ожидается так и вовсе ниже 3%. Впрочем, на ряд продовольственных товаров цены будут расти гораздо более высокими темпами. «Газета.Ru» разбиралась, какие продукты подорожают в новом году гораздо сильнее других. В 2019 году стоимость молока-сырья выросла на 9% и, как следствие, увеличилась цена на готовую продукцию: питьевое молоко за год подорожало на 7%, кефир, ряженка, творог — на 6 %, йогурт – на 8%, сметана на 7%, мороженое на 9%, полутвердые сыры на 10%, сливочное масло – на 11%, рассказала «Газете.Ru» заместитель председателя комитета МТПП по развитию предпринимательства в АПК, генеральный директор «Petrova 5 Consulting» Марина Петрова. По ее словам, в 2020 году стоит ожидать дальнейшего увеличения цен на молочные продукты. Темпы повышения, по ее словам, будут сопоставимы с ростом в 2019 году. «Производители плавно поднимают цены, потому что беспокоятся о возможном снижении объемов прода

In [16]:
inputs = tokenizer(text_to_summarize, max_length=MAX_INPUT_LENGTH, truncation=True, return_tensors="pt")

In [17]:
output = model.generate(**inputs, min_length=10, max_length=32)

In [18]:
output

tensor([[    0,   259,  6825,   310,  8745,  2093,   315,   972,  5546,   259,
          4466,   259, 16554,     1]])

In [19]:
tokenizer.batch_decode(output, skip_special_tokens=True)[0]

'Цена на молоко в 2020 году будет расти'

## HNY

<img src="attachements/third/hny_meme.jpeg" width="400"/>

## Bye

<img src="attachements/second/thanks_attention.png" width="1200"/>